In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # Newer versions


## Read Files and Preprocessing

In [2]:
whole_data = pd.read_csv(r"data/recipe_train.csv", index_col = False, delimiter = ',', header=0)

# Split into attributes and class
y_df = whole_data["duration_label"]    #class df
x_df = whole_data.drop(['duration_label'], axis=1)    # features df

# train test split
X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size = 0.33, random_state = 123)
print('X_train: {} X_test: {}'.format(X_train.shape, X_test.shape))

X_train: (26800, 5) X_test: (13200, 5)


In [ ]:
# Using Naive Bayes:
# Need Numeric Data ONLYs

import sklearn.naive_bayes as nb
##print(dir(nb))
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

gnb_accs = []
mnb_accs = []
bnb_accs = []
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

for i in range(3):
    X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.33, random_state=i)
    gnb.fit(X_train, y_train)
    acc = gnb.score(X_test, y_test)
    print("GNB score %f " %acc)
    gnb_accs.append(acc)
    
    mnb.fit(X_train, y_train)
    acc = mnb.score(X_test, y_test)
    print("MNB score %f " %acc)
    mnb_accs.append(acc)
    
    bnb.fit(X_train, y_train)
    acc = bnb.score(X_test, y_test)
    print("BNB score %f " %acc)
    bnb_accs.append(acc)
    
print('Avg GNB score: {}'.format(np.mean(gnb_accs)))
print('Avg MNB score: {}'.format(np.mean(mnb_accs)))
print('Avg BNB score: {}'.format(np.mean(bnb_accs)))


# 
# Bibliograhy:
# 

Generating Personalized Recipes from Historical User Preferences. Bodhisattwa Prasad Majumder,
Shuyang Li, Jianmo Ni, Julian McAule+y, in Proceedings of the 2019 Conference on Empirical
Methods in Natural Language Processing and the 9th International Joint Conference on Natural
Language Processing (EMNLP-IJCNLP), 2019.